# Extraction of SWR from CA1 recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [ ]:
cd "C:/Users/AudreyHay/Documents/script"

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
import fnmatch

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

%matplotlib widget


# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

In [ ]:
dpath = "C:/Users/AudreyHay/Documents/"
#dpath = "//10.69.168.1/crnldata/waking/audrey_hay/"
try:
    %store -r dpath
except:
    print("data not in store")
    #dpath ="//10.69.168.1/crnldata/waking/audrey_hay/"
    dpath = "C:/Users/AudreyHay/Documents/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

### To change (only if 'sommeil' Bonsaï file)

In [ ]:
mice='Blue'    # mice='Pink' OR  mice='Blue'

## Load LFPs file

In [ ]:
folderpath = Path(dpath)

# Load channels map 
Channels = 'SoleneMiceLFPChannels.xlsx'
#Channels = '//10.69.168.1/crnldata/waking/audrey_hay/CamilleData/SoleneMiceLFPChannels.xlsx'  
allchannels = pd.read_excel(Channels)
Map=np.array(allchannels['MICE'])

ctx=7 # ctx=7 for CA1, ctx=6 for mEC, ctx=9 for RSP, ctx=10 for Perirhinal 
emg= 0

############
## Bonsai ##
############

if find_files_with_string(folderpath,  ".bin"): 
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype=np.uint16)
    All = All - int(65535/2)
    All = All.astype(np.int16)
    
    ### Sommeil ###
    if 'sommeil' in dpath:
        numchannel=64
        All = All.reshape(-1,numchannel)      
        Ch=32 if mice == 'Pink' else 0

        EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch #EMG= 1st line in the excel file
        EMG  =  All[:, EMGch]

        try:
            CA1ch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch     #CA1 == 8th line in the excel file
            CA1ch2=int(allchannels[mice][ctx].split(',')[1]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch
            CA1  =  All[:, CA1ch1]-All[:, CA1ch2]
        except:
            CA1ch1=int(allchannels[mice][ctx]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch
            print('/!\ One CA1 channel only')
            CA1  =  All[:, CA1ch1] 
        print(f'Bonsaï file loaded: {mice} mouse / sommeil file')
    
    ### Cheeseboard ###
    elif 'cheeseboard' in dpath:
        numchannel=32
        All = All.reshape(-1,numchannel)
        Ch=32 if 'pink' in dpath  else 0
        mice='Pink' if 'pink' in dpath  else 'Blue'      

        EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch #EMG= 1st line in the excel file
        EMG  =  All[:, EMGch]
        try:
            CA1ch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch     #CA1 == 8th line in the excel file
            CA1ch2=int(allchannels[mice][ctx].split(',')[1]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch
            CA1  =  All[:, CA1ch1]-All[:, CA1ch2]
        except:
            CA1ch1=int(allchannels[mice][ctx]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch
            print('/!\ One CA1 channel only')
            CA1  =  All[:, CA1ch1]         
        print(f'Bonsaï file loaded: {mice} mouse / cheeseboard file')
        
###############
## OpenEphys ##
###############

elif find_files_with_string(folderpath,  "continuous.dat"): 
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype="int16")
    numchannel=64
    All = All.reshape(-1,numchannel)
    All = np.memmap(matching_file, mode='r', dtype='int16')
    All = All.reshape(-1, numchannel)     
    print('Open Ephy file loaded: continuous.dat')
    Ch=32 if 'pink' in dpath  else 0
    mice='Pink' if 'pink' in dpath  else 'Blue'

    EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch #EMG= 1st line in the excel file
    CA1ch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch     #CA1 == 8th line in the excel file
    CA1ch2=CA1ch1 if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch       
    
    EMG  =  All[:, EMGch]
    CA1  =  All[:, CA1ch1] if mice == 'Blue'else All[:, CA1ch1]-All[:, CA1ch2]

## OpenEphys Gaelle's Data ##
elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): 
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    All = All.reshape(-1,32)
    print('Open Ephy file loaded: RawDataChannelExtractedDS.npy')


# Remove Wake signal

In [ ]:
# Filter parameter :
f_lowcut = 200.
f_hicut = 400.
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filter creation :
b, a = signal.butter(N, Wn, 'band')
filt_EMG = signal.filtfilt(b, a, EMG)

# Plot
times = np.arange(0, EMG.size/fs, 1./fs)
timesmin = np.arange(0, EMG.size/fs/60, 1./fs/60)

In [ ]:
# Parameter and computation of CWT
w = 4.
freq = np.linspace(200, 400, 50)
widths = w*fs / (2*freq*np.pi)
EMGcwt = signal.cwt(EMG, signal.morlet2, widths, w=w)

# Projection calculation
absEMGcwt = np.absolute(EMGcwt)
proj_EMGcwt = np.sum(absEMGcwt, axis = 0)/50
mproj_EMGcwt = np.mean(proj_EMGcwt)
sdproj_EMGcwt = np.std(proj_EMGcwt)
sd3proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt*3
sd05proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt*0.5
sd1proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt


In [ ]:
# Assigning values wake (1, 2) and sleep (0)
numpnts = EMG.size
EMGstatusRaw = np.zeros(numpnts)
for ind in range(numpnts):
    if proj_EMGcwt[ind]<sd05proj_EMGcwt:
        EMGstatusRaw[ind] = 0
    elif proj_EMGcwt[ind]>sd3proj_EMGcwt:
        EMGstatusRaw[ind] = 2
    else:
        EMGstatusRaw[ind] = 1

# Expanding borders for wake (1, 2) and sleep (0) to ±1 s around detected muscular activity
EMGstatusRaw2 = np.zeros(numpnts)
for ind in range(numpnts):
    if EMGstatusRaw[ind]>1:
       EMGstatusRaw2[ind-1000:ind+1000] = 2
    elif EMGstatusRaw[ind]==1:
        for ind2 in range(ind-1000, ind+1000):
            if ind2==numpnts:
                break
            elif EMGstatusRaw2[ind2]<2:
                EMGstatusRaw2[ind2] = 1

In [ ]:
EMGStatusBoolLib = (EMGstatusRaw2>1)
EMGStatusBoolCons = (EMGstatusRaw2>0)

LFP = All[:,:]

LFPwake0 = LFP.copy()
LFPwake0[EMGStatusBoolLib,:] = 0

LFPwakeremoved = LFP.copy()
LFPwakeremoved = LFPwakeremoved[~EMGStatusBoolLib, :]

Wake removed from CA1 LFP

In [ ]:
try: 
    CA1wakeremoved = LFPwakeremoved[:,CA1ch1]-LFPwakeremoved[:,CA1ch2] 
except: 
    CA1wakeremoved = LFPwakeremoved[:,CA1ch1]

# Band pass filter
        SWR: 120-200 Hz

In [ ]:
# Filtre parameter:
f_lowcut = 120.
f_hicut = 200.
fs = 1000
nyq = 0.5 * fs
N = 6                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)


## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [ ]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(120, 200, 80)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation
absCA1NWcwt = np.absolute(CA1NWcwt)
proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/80
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
#sd3proj_CA1cwt = sdproj_CA1cwt*3
#sd10proj_CA1cwt = sdproj_CA1cwt*10
#sd8proj_CA1cwt = sdproj_CA1cwt*8
#sd7proj_CA1cwt = sdproj_CA1cwt*7
sd6proj_CA1cwt = sdproj_CA1cwt*6

Second on the signal for which wake times have been zeroed

In [ ]:
# Conservative boolean filtering of CA1 filtered signal
BooleanCons = EMGStatusBoolCons
fCA1wake0C = filt_CA1.copy()
fCA1wake0C[BooleanCons] = 0
CA1wake0C = CA1.copy()
CA1wake0C[BooleanCons] = 0
# Liberal boolean filtering of CA1 filtered signal
BooleanLib = EMGStatusBoolLib
fCA1wake0L = filt_CA1.copy()
fCA1wake0L[BooleanLib] = 0
CA1wake0L = CA1.copy()
CA1wake0L[BooleanLib] = 0

# Computation of CWT
CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCA1W0Ccwt = np.absolute(CA1cwtWake0cons)
proj_CA1W0Ccwt = np.sum(absCA1W0Ccwt, axis = 0)/80
absCA1W0Lcwt = np.absolute(CA1cwtWake0lib)
proj_CA1W0Lcwt = np.sum(absCA1W0Lcwt, axis = 0)/80


## Extracting SWRs and determining main properties 

First extraction of SWR peaks, initiation, end and width

In [ ]:
# 10 sd threshold
peaks, properties = find_peaks(proj_CA1W0Lcwt, prominence=1, width=20, height=sd6proj_CA1cwt) #AB detection with 8*SD // Audrey's detection=3*SD
properties["prominences"], properties["widths"]

# SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Display subset

Second extraction of main frequency and power 

In [ ]:
projMaxP_cwtmg = np.max(CA1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CA1cwtWake0lib, axis = 0) + 120
projMaxP_cwtmg.shape

nb_SWR = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_SWR:
    SWR_start = int(SWR_prop[3,tt])
    SWR_stop = int(SWR_prop[4,tt])
    SWR_MaxP = projMaxP_cwtmg[SWR_start:SWR_stop]
    SWR_MaxF = projMaxF_cwtmg[SWR_start:SWR_stop]
    data[tt, 0] = max(SWR_MaxF).round()
    data[tt, 1] = max(SWR_MaxP).round()
    data[tt, 2] = round(sum(SWR_MaxF)/len(SWR_MaxF))
    data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))

param_SWR = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSWR_prop = SWR_prop.transpose()
pd_prop_SWR = pd.DataFrame(tSWR_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_SWR = pd.concat([pd_prop_SWR, param_SWR], axis=1)

SWR_peak = peaks
SWR_start = SWR_prop[3,:].astype(int)
SWR_end = SWR_prop[4,:].astype(int)

### Store the results in All_SWR_prop pd dataframe and save as pkl/csv for post processing.

In [ ]:
filename = folderpath / f'SWRproperties_{mice}_{Map[ctx]}Initial_AB.xlsx'
All_SWR.to_excel(filename)

filename = folderpath / f'SWRproperties_{mice}_{Map[ctx]}_AB.xlsx'
All_SWR.to_excel(filename)

### Display

ephys viewer to check SWR detection

In [ ]:
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: SWR_peak, 1: SWR_start, 2: SWR_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}
combined = np.stack([CA1, filt_CA1*10, proj_CA1W0Ccwt*10, proj_CA1W0Lcwt*10], axis = 1)

source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel

view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

#view1.by_channel_params['ch0', 'offset'] = 0
#view1.by_channel_params['ch1', 'offset'] = -1
#view1.by_channel_params['ch2', 'offset'] = -1
#view1.by_channel_params['ch3', 'offset'] = -1

#view1.by_channel_params['ch0', 'gain'] = .001
#view1.by_channel_params['ch1', 'gain'] = .1
#view1.by_channel_params['ch2', 'gain'] = .1
#view1.by_channel_params['ch3', 'gain'] = .1

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

Quick plot of average and max intensity (can be done with freq as well) paired.